In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pathlib import Path
from shutil import move
import whisper
from resemblyzer import VoiceEncoder, preprocess_wav
from resemblyzer.hparams import sampling_rate
import numpy as np
import os
import librosa
from pydub import AudioSegment
from sklearn.cluster import KMeans
from scipy import stats
import pandas as pd

# Create output folders
os.makedirs("all_audio", exist_ok=True)
os.makedirs("all_csv", exist_ok=True)

rang = [
    673, 677, 680, 684, 692, 695, 697, 702, 703, 707
]

model = whisper.load_model("medium")

for i in range(300,719,1):#rang:
  try:
    pid = f"{i}_P"
    archive = f"{pid}.tar.gz"
    url = f"https://dcapswoz.ict.usc.edu/wwwedaic/data/{archive}"

    print(f"\n📥 Downloading: {archive}")
    os.system(f"wget -q --show-progress {url}")

    print(f"📦 Extracting audio and CSV from: {archive}")
    os.system(f"tar --wildcards -xzf {archive} '{pid}/*AUDIO.wav'")

    print(f"🧹 Removing archive: {archive}")
    os.system(f"rm {archive}")

    # Move the extracted files to organized folders
    audio_path = Path(f"{pid}/{pid.replace('_P','')}_AUDIO.wav")

    if audio_path.exists():
        move(str(audio_path), f"all_audio/{audio_path.name}")

    # Optional: delete the now-empty folder
    os.system(f"rm -rf {pid}")

    print(i)
  # Load and preprocess audio
    try:
      wav_fpath = "/content/all_audio/"+str(i)+"_AUDIO.wav"
      wav, _ = librosa.load(wav_fpath, sr=sampling_rate)
    except:
      continue
    # Create voice embeddings every second
    encoder = VoiceEncoder()
    embed_frames = encoder.embed_utterance(wav, return_partials=True, rate=1)

    # Cluster embeddings
    n_speakers = 2  # adjust based on your case
    kmeans = KMeans(n_clusters=n_speakers).fit(embed_frames[1])
    speaker_labels = kmeans.labels_

    # Load Whisper model for transcription
    result = model.transcribe(wav_fpath)

    # Prepare combined speaker-labeled transcription
    transcript = []
    for j, segment in enumerate(result['segments']):
        start = segment['start']
        end = segment['end']
        text = segment['text'].strip()

        # Approximate speaker label for this time segment
        # Use the label of the center of the segment
        center_time = int((start + end) / 2)

        speaker_id = stats.mode(speaker_labels[center_time-1:center_time+1]) if center_time < len(speaker_labels) else 0

        try:
          transcript.append({
              "Timestamp": f"{start:.2f}–{end:.2f}",
              "Transcript": text,
              "Speaker": f"Speaker {int(speaker_id[0]) + 1}"
          })
        except:
          pass

    # Save to CSV
    import pandas as pd
    df = pd.DataFrame(transcript)
    df.to_csv("/content/drive/MyDrive/all_speakers/"+str(i)+"_transcript.csv", index=False)
    print("Saved as voice_clustered_transcript.csv")


  except:
    print(i)
    pass


100%|█████████████████████████████████████| 1.42G/1.42G [00:27<00:00, 55.5MiB/s]



📥 Downloading: 300_P.tar.gz
📦 Extracting audio and CSV from: 300_P.tar.gz
🧹 Removing archive: 300_P.tar.gz
300
Loaded the voice encoder model on cpu in 0.03 seconds.


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


300

📥 Downloading: 301_P.tar.gz
📦 Extracting audio and CSV from: 301_P.tar.gz
🧹 Removing archive: 301_P.tar.gz
301

📥 Downloading: 302_P.tar.gz


/tmp/ipython-input-3-3883333130.py:53: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, _ = librosa.load(wav_fpath, sr=sampling_rate)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


📦 Extracting audio and CSV from: 302_P.tar.gz
🧹 Removing archive: 302_P.tar.gz
302
Loaded the voice encoder model on cpu in 0.03 seconds.


In [ ]:
import zipfile

drive_audio = "/content/all_audio"
# Zip audio files
audio_zip_path = "/content//all_audio.zip"
with zipfile.ZipFile(audio_zip_path, 'w') as audio_zip:
    for file in os.listdir(drive_audio):
        file_path = os.path.join(drive_audio, file)
        audio_zip.write(file_path, arcname=file)

print("✅ Zipping complete!")


NameError: name 'os' is not defined

In [ ]:
!pip install openai-whisper
!pip install pydub
!pip install google-generativeai
!pip install tensorflow
!pip install opensmile soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 941.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.9 MB/

In [ ]:
!pip install openai-whisper
!pip install pydub
!pip install google-generativeai
!pip install tensorflow
!pip install opensmile soundfile
!pip install pyannote.audio
!pip install resemblyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.8 MB/s

In [ ]:
import whisper
from resemblyzer import VoiceEncoder, preprocess_wav
from resemblyzer.hparams import sampling_rate
import numpy as np
import os
import librosa
from pydub import AudioSegment
from sklearn.cluster import KMeans
from scipy import stats
import pandas as pd

# model = whisper.load_model("medium")

for i in range(300,719,1):
  print(i)
# Load and preprocess audio
  try:
    wav_fpath = "/content/all_audio/"+str(i)+"_AUDIO.wav"
    wav, _ = librosa.load(wav_fpath, sr=sampling_rate)
  except:
    continue
  # Create voice embeddings every second
  encoder = VoiceEncoder()
  embed_frames = encoder.embed_utterance(wav, return_partials=True, rate=1)

  # Cluster embeddings
  n_speakers = 2  # adjust based on your case
  kmeans = KMeans(n_clusters=n_speakers).fit(embed_frames[1])
  speaker_labels = kmeans.labels_

  # Load Whisper model for transcription
  result = model.transcribe(wav_fpath)

  # try:
  #   result = pd.read_csv("/content/transcripts/"+str(i)+"_transcript.csv")
  # except:
  #   continue

  # Prepare combined speaker-labeled transcription
  transcript = []
  for j, segment in enumerate(result['segments']):
      start = segment['start']
      end = segment['end']
      text = segment['text'].strip()

  # for j in range(result.shape[0]):
  #     start = result['start'].iloc[j]
  #     end = result['end'].iloc[j]
  #     text = result['text'].iloc[j].strip()

      # Approximate speaker label for this time segment
      # Use the label of the center of the segment
      center_time = int((start + end) / 2)

      speaker_id = stats.mode(speaker_labels[center_time-1:center_time+1]) if center_time < len(speaker_labels) else 0

      try:
        transcript.append({
            "Timestamp": f"{start:.2f}–{end:.2f}",
            "Transcript": text,
            "Speaker": f"Speaker {int(speaker_id[0]) + 1}"
        })
      except:
        pass

  # Save to CSV
  import pandas as pd
  df = pd.DataFrame(transcript)
  df.to_csv("/content/all_speakers/"+str(i)+"_transcript.csv", index=False)
  print("Saved as voice_clustered_transcript.csv")



300
Loaded the voice encoder model on cuda in 0.02 seconds.


/tmp/ipython-input-3-3128896435.py:55: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  speaker_id = stats.mode(speaker_labels[center_time-1:center_time+1]) if center_time < len(speaker_labels) else 0


Saved as voice_clustered_transcript.csv
301
Loaded the voice encoder model on cuda in 0.01 seconds.
Saved as voice_clustered_transcript.csv
302
Loaded the voice encoder model on cuda in 0.01 seconds.
Saved as voice_clustered_transcript.csv
303
Loaded the voice encoder model on cuda in 0.02 seconds.
Saved as voice_clustered_transcript.csv
304
Loaded the voice encoder model on cuda in 0.02 seconds.
Saved as voice_clustered_transcript.csv
305
Loaded the voice encoder model on cuda in 0.01 seconds.
Saved as voice_clustered_transcript.csv
306
Loaded the voice encoder model on cuda in 0.02 seconds.
Saved as voice_clustered_transcript.csv
307
Loaded the voice encoder model on cuda in 0.01 seconds.


In [ ]:
# import os
# import shutil

# dir_path = '/content/all_speakers'

# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
#     print(f"Deleted: {dir_path}")
# else:
#     print(f"Directory not found: {dir_path}")

OSError: [Errno 125] Operation canceled: '/content/drive/MyDrive'

In [ ]:
import zipfile

drive_audio = "/content/all_speakers"
# Zip audio files
audio_zip_path = "/content/all_transcripts.zip"
with zipfile.ZipFile(audio_zip_path, 'w') as audio_zip:
    for file in os.listdir(drive_audio):
        file_path = os.path.join(drive_audio, file)
        audio_zip.write(file_path, arcname=file)

print("✅ Zipping complete!")


✅ Zipping complete!


In [ ]:
import whisper
from pydub import AudioSegment
import google.generativeai as genai
from datetime import timedelta
import os, glob, zipfile
import pandas as pd
import opensmile

In [ ]:
model_whisper = whisper.load_model("small")

def transcribe_audio(audio_path):
    result = model_whisper.transcribe(audio_path)
    return result["segments"]  # Each has 'text', 'start', 'end'

def clean_transcription(segment_text):
    prompt = (
        "The following transcription has possible mistakes and repeated phrases. "
        "Please correct transcription errors and remove duplicate content:\n\n"
        f"\"{segment_text}\"\n\n"
        "Return only the cleaned text."
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def extract_segment(audio_path, start_sec, end_sec, output_path):
    audio = AudioSegment.from_wav(audio_path)
    segment = audio[start_sec * 1000 : end_sec * 1000]  # milliseconds
    segment.export(output_path, format="wav")

In [ ]:
import csv
import os

def process_audio_pipeline(audio_path, output_csv_path="segments.csv"):
    segments = transcribe_audio(audio_path)

    # Open CSV file for writing
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['text', 'start', 'end'])
        writer.writeheader()

        for i, seg in enumerate(segments):
            writer.writerow({
                'text': seg['text'],
                'start': seg['start'],
                'end': seg['end']
            })

    print(f"\n CSV saved at: {output_csv_path}")

In [ ]:
import os

def batch_process_audio(folder_path, output_folder="/content/drive/MyDrive/CGS_Audio_FacialData/transcripts"):
    os.makedirs(output_folder, exist_ok=True)

    for i in range(300, 719):  # 719 because range is exclusive on the upper end
        file_name = f"all_audio/{i}_AUDIO.wav"
        file_path = os.path.join(folder_path, file_name)

        if os.path.exists(file_path):
            output_csv = os.path.join(output_folder, f"{i}_transcript.csv")
            try:
                print(f"Processing {file_name}...")
                process_audio_pipeline(file_path, output_csv_path=output_csv)
            except Exception as e:
                print(f" Error processing {file_name}: {e}")
        else:
            print(f" File not found: {file_name}")

# 🔧 Usage
batch_process_audio("/content/")

 File not found: all_audio/300_AUDIO.wav
 File not found: all_audio/301_AUDIO.wav
 File not found: all_audio/302_AUDIO.wav
 File not found: all_audio/303_AUDIO.wav
 File not found: all_audio/304_AUDIO.wav
 File not found: all_audio/305_AUDIO.wav
 File not found: all_audio/306_AUDIO.wav
 File not found: all_audio/307_AUDIO.wav
 File not found: all_audio/308_AUDIO.wav
 File not found: all_audio/309_AUDIO.wav
 File not found: all_audio/310_AUDIO.wav
 File not found: all_audio/311_AUDIO.wav
 File not found: all_audio/312_AUDIO.wav
 File not found: all_audio/313_AUDIO.wav
 File not found: all_audio/314_AUDIO.wav
 File not found: all_audio/315_AUDIO.wav
 File not found: all_audio/316_AUDIO.wav
 File not found: all_audio/317_AUDIO.wav
 File not found: all_audio/318_AUDIO.wav
 File not found: all_audio/319_AUDIO.wav
 File not found: all_audio/320_AUDIO.wav
 File not found: all_audio/321_AUDIO.wav
 File not found: all_audio/322_AUDIO.wav
 File not found: all_audio/323_AUDIO.wav
 File not found:

In [ ]:
import os
from pathlib import Path
from shutil import move

# Create output folders
drive_csv = "/content/drive/MyDrive/CGS_Audio_FacialData/all_fau"
os.makedirs(drive_csv, exist_ok=True)

rang = [
    300, 301, 306, 317, 320, 321, 331, 334, 336, 343, 344, 347, 350,
    365, 371, 373, 374, 381, 382, 388, 393, 401, 402, 408, 412, 415,
    423, 425, 431, 433, 435, 437, 441, 442, 448, 451, 454, 455, 465,
    468, 473, 475, 479, 480, 484, 486, 617, 627, 632, 653, 657, 667,
    670, 687, 698, 713,
    600, 602, 604, 605, 606, 607, 609, 615, 618, 619, 620, 622, 623,
    624, 625, 626, 629, 631, 634, 635, 636, 637, 638, 640, 649, 650,
    651, 652, 655, 656, 658, 659, 661, 663, 664, 666, 669, 676, 679,
    682, 683, 688, 689, 691, 693, 696, 699, 705, 708, 709, 710, 712,
    715, 716, 717, 718,
    673, 677, 680, 684, 692, 695, 697, 702, 703, 707
]


for i in rang:
    pid = f"{i}_P"
    archive = f"{pid}.tar.gz"
    url = f"https://dcapswoz.ict.usc.edu/wwwedaic/data/{archive}"

    print(f"\n📥 Downloading: {archive}")
    os.system(f"wget -q --show-progress {url}")

    print(f"📦 Extracting audio and CSV from: {archive}")
    os.system(f"tar --wildcards -xzf {archive} '{pid}/features/*OpenFace2.1.0_Pose_gaze_AUs.csv'")

    print(f"🧹 Removing archive: {archive}")
    os.system(f"rm {archive}")

    # Move the extracted files to organized folders
    csv_path = Path(f"{pid}/features/{pid.replace('_P','')}_OpenFace2.1.0_Pose_gaze_AUs.csv")

    if csv_path.exists():
        move(str(csv_path), f"{drive_csv}/{csv_path.name}")

    # Optional: delete the now-empty folder
    os.system(f"rm -rf {pid}")



📥 Downloading: 300_P.tar.gz
📦 Extracting audio and CSV from: 300_P.tar.gz
🧹 Removing archive: 300_P.tar.gz

📥 Downloading: 301_P.tar.gz
📦 Extracting audio and CSV from: 301_P.tar.gz
🧹 Removing archive: 301_P.tar.gz

📥 Downloading: 306_P.tar.gz
📦 Extracting audio and CSV from: 306_P.tar.gz
🧹 Removing archive: 306_P.tar.gz

📥 Downloading: 317_P.tar.gz
📦 Extracting audio and CSV from: 317_P.tar.gz
🧹 Removing archive: 317_P.tar.gz

📥 Downloading: 320_P.tar.gz
📦 Extracting audio and CSV from: 320_P.tar.gz
🧹 Removing archive: 320_P.tar.gz

📥 Downloading: 321_P.tar.gz
📦 Extracting audio and CSV from: 321_P.tar.gz
🧹 Removing archive: 321_P.tar.gz

📥 Downloading: 331_P.tar.gz
📦 Extracting audio and CSV from: 331_P.tar.gz
🧹 Removing archive: 331_P.tar.gz

📥 Downloading: 334_P.tar.gz
📦 Extracting audio and CSV from: 334_P.tar.gz
🧹 Removing archive: 334_P.tar.gz

📥 Downloading: 336_P.tar.gz
📦 Extracting audio and CSV from: 336_P.tar.gz
🧹 Removing archive: 336_P.tar.gz

📥 Downloading: 343_P.tar.gz

In [ ]:
import pandas as pd

# Paths to your CSV files
file1 = '/content/618_BoAW_openSMILE_2.3.0_MFCC.csv'
file2 = '/content/618_BoAW_openSMILE_2.3.0_eGeMAPS.csv'

# Load the CSV files (semicolon-separated)
df1 = pd.read_csv(file1, sep=';', header = None)
df2 = pd.read_csv(file2, sep=';', header = None)

df1 = df1.iloc[:, 1:]
df2 = df2.iloc[:, 2:]

# Concatenate along columns (side by side)
merged_df = pd.concat([df1, df2], axis=1)

# Save to a new CSV file
output_file = '/content/drive/MyDrive/Feature_Datasets/BoAW_combined/618_BoAW_openSMILE_2.3.0_combined.csv'
merged_df.to_csv(output_file, index=False, header=None)

print(f"Merged CSV saved to: {output_file}")


Merged CSV saved to: /content/drive/MyDrive/Feature_Datasets/BoAW_combined/618_BoAW_openSMILE_2.3.0_combined.csv


In [ ]:
import shutil

# Move a single file
shutil.move('/content/691_BoAW_openSMILE_2.3.0_combined.csv', '/content/drive/MyDrive/Feature_Datasets/BoAW_combined/691_BoAW_openSMILE_2.3.0_combined.csv')


'/content/drive/MyDrive/Feature_Datasets/BoAW_combined/691_BoAW_openSMILE_2.3.0_combined.csv'

In [ ]:
import shutil

# Folder to zip
folder_path = '/content/drive/MyDrive/CGS_Audio_FacialData/all_fau'

# Output zip file path (without .zip extension)
zip_path = '/content/drive/MyDrive/CGS_Audio_FacialData/all_fau.zip'

# Create the zip archive
shutil.make_archive(zip_path, 'zip', folder_path)


'/content/drive/MyDrive/CGS_Audio_FacialData/all_fau.zip.zip'

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
transcript_path = '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts'
audio_path = '/content/drive/MyDrive/Feature_Datasets/BoAW_combined'

In [ ]:
UTTERANCES_PER_CHUNK = 12
OVERLAP = 6
MAX_CHUNKS = 87

In [ ]:
audio_data = []

transcript_path = '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts'
fau_path = '/content/drive/MyDrive/Feature_Datasets/BoAW_combined'

for person_id in range(300, 719):
    try:
        transcript_file = os.path.join(transcript_path, f'{person_id}_transcript.csv')
        fau_file_1 = os.path.join(fau_path, f'{person_id}_BoAW_openSMILE_2.3.0_combined.csv')

        transcript_df = pd.read_csv(transcript_file)
        fau_df_1 = pd.read_csv(fau_file_1, header=None)  # Assuming no column names

        chunks = []
        utterance_features = []

        for _, row in transcript_df.iterrows():
            start, end = row['start'], row['end']

            # Select segment between start and end time
            segment_1 = fau_df_1[(fau_df_1.iloc[:, 0] >= start) & (fau_df_1.iloc[:, 0] <= end)]

            feature_cols = fau_df_1.shape[1] - 1  # exclude timestamp
            if segment_1.empty:
                pooled_1 = np.zeros(feature_cols)
            else:
                pooled_1 = segment_1.iloc[:, 1:].max().values  # max over feature columns only

            utterance_features.append(pooled_1)

        # Chunking with overlap
        for i in range(0, len(utterance_features) - UTTERANCES_PER_CHUNK + 1, UTTERANCES_PER_CHUNK - OVERLAP):
            chunk = np.vstack(utterance_features[i:i + UTTERANCES_PER_CHUNK])
            chunk_pooled = chunk.max(axis=0)
            chunks.append(chunk_pooled)

        # Padding to MAX_CHUNKS
        while len(chunks) < MAX_CHUNKS:
            chunks.append(np.zeros(feature_cols))
        chunks = chunks[:MAX_CHUNKS]

        audio_data.append(np.array(chunks))

    except Exception as e:
        print(f"Error with {person_id}: {e}")


Error with 342: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/342_transcript.csv'
Error with 394: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/394_transcript.csv'
Error with 398: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/398_transcript.csv'
Error with 460: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/460_transcript.csv'
Error with 493: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/493_transcript.csv'
Error with 494: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/494_transcript.csv'
Error with 495: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/495_transcript.csv'
Error with 496: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_Fac

In [ ]:
!unzip /content/drive/MyDrive/CGS_Audio_FacialData/all_csv_train.zip -d /content/

Archive:  /content/drive/MyDrive/CGS_Audio_FacialData/all_csv_train.zip
 extracting: /content/302_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/303_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/304_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/305_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/307_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/308_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/309_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/310_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/311_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/312_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/313_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/314_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/315_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/316_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/318_OpenFace2.1.0_Pose_gaze_AUs.csv  
 extracting: /content/319_Op

In [ ]:
!unzip /content/drive/MyDrive/CGS_Audio_FacialData/all_fau.zip.zip -d /content/

Archive:  /content/drive/MyDrive/CGS_Audio_FacialData/all_fau.zip.zip
  inflating: /content/331_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/334_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/336_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/343_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/344_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/347_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/350_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/365_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/371_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/373_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/374_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/381_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/382_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/388_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/393_OpenFace2.1.0_Pose_gaze_AUs.csv  
  inflating: /content/401_Open

In [ ]:
FAU_FEATURE_COLS = ['frame', 'confidence', 'success']

In [ ]:
video_data = []

transcript_path = '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts'
fau_path = '/content/'

for person_id in range(300, 719):
    try:
        transcript_file = os.path.join(transcript_path, f'{person_id}_transcript.csv')
        fau_file = os.path.join(fau_path, f'{person_id}_OpenFace2.1.0_Pose_gaze_AUs.csv')

        transcript_df = pd.read_csv(transcript_file)

        # Only load timestamp + FAU features
        # Get all columns
        all_columns = pd.read_csv(fau_file, nrows=0).columns.tolist()

        # Keep everything except the ones in FAU_FEATURE_COLS
        columns_to_use = [col for col in all_columns if col not in FAU_FEATURE_COLS]

        # Load only the selected columns
        fau_df = pd.read_csv(fau_file, usecols=columns_to_use)


        chunks = []
        utterance_features = []

        for _, row in transcript_df.iterrows():
            start, end = row['start'], row['end']
            segment = fau_df[(fau_df['timestamp'] >= start) & (fau_df['timestamp'] <= end)]
            feature_columns = [col for col in fau_df.columns if col != 'timestamp']

            if segment.empty:
                pooled = np.zeros(len(feature_columns))
            else:
                pooled = segment[feature_columns].max().values

            utterance_features.append(pooled)

        # Chunking with overlap
        for i in range(0, len(utterance_features) - UTTERANCES_PER_CHUNK + 1, UTTERANCES_PER_CHUNK - OVERLAP):
            chunk = np.vstack(utterance_features[i:i+UTTERANCES_PER_CHUNK])
            chunk_pooled = chunk.max(axis=0)
            chunks.append(chunk_pooled)

        # Padding
        while len(chunks) < MAX_CHUNKS:
            chunks.append(np.zeros(len(feature_columns)))
        chunks = chunks[:MAX_CHUNKS]

        video_data.append(np.array(chunks))

    except Exception as e:
        print(f"Error with {person_id}: {e}")


Error with 342: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/342_transcript.csv'
Error with 394: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/394_transcript.csv'
Error with 398: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/398_transcript.csv'
Error with 460: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/460_transcript.csv'
Error with 493: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/493_transcript.csv'
Error with 494: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/494_transcript.csv'
Error with 495: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts/495_transcript.csv'
Error with 496: [Errno 2] No such file or directory: '/content/drive/MyDrive/CGS_Audio_Fac

In [ ]:
import os
import pandas as pd

# Assuming the 'transcripts' folder contains segmented CSV files for each participant
transcript_folder = '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts'
transcripts_data = {}

# Load each CSV file into a dictionary
for participant_id in range(300, 719):
    file_path = os.path.join(transcript_folder, f"{participant_id}_transcript.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)  # Load the CSV
        transcripts_data[participant_id] = df['text'].tolist()  # Get list of text utterances

In [ ]:
def chunk_transcript(utterances, chunk_size=10, overlap=4):
    step = chunk_size - overlap
    chunks = []
    for i in range(0, len(utterances) - chunk_size + 1, step):
        chunk = utterances[i:i + chunk_size]
        chunks.append(" ".join(chunk))  # Join the chunks into a single string
    return chunks

# Apply chunking to each participant's transcript
chunks_data = {}
for participant_id, utterances in transcripts_data.items():
    chunks_data[participant_id] = chunk_transcript(utterances)

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def get_embedding(text):
    # Generate embeddings for the input text
    return model.encode(text).tolist()  # Convert numpy array to list

# Embed all chunks for each participant
embeddings_data = {}
for participant_id, chunks in chunks_data.items():
    embeddings_data[participant_id] = [get_embedding(chunk) for chunk in chunks]

In [ ]:
# Create a list of PTSD severity scores and corresponding embeddings
text_data = []  # Store embeddings

for participant_id, embeddings in embeddings_data.items():
    text_data.append(embeddings)  # Add embeddings

In [ ]:
audio_data = np.array(audio_data)
audio_data.shape

(275, 87, 200)

In [ ]:
video_data = np.array(video_data)
video_data.shape

(275, 87, 49)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
embedding_dim = len(text_data[0])  # e.g., 1536 for `text-embedding-small`

# Pad sequences to ensure consistent input size
MAX_CHUNKS = 87
text_data = pad_sequences(text_data, padding='post', dtype='float32', maxlen=MAX_CHUNKS)

# Convert to numpy arrays
text_data = np.array(text_data)

In [ ]:
text_data.shape

(275, 87, 384)

In [ ]:
X_all = np.concatenate([audio_data, text_data, video_data], axis=2)

In [ ]:
import os
import numpy as np
import pandas as pd

transcript_path = '/content/drive/MyDrive/CGS_Audio_FacialData/transcripts'

rows = []

# Track current index into X_all
sample_idx = 0

for person_id in range(300, 719):
    transcript_file = os.path.join(transcript_path, f'{person_id}_transcript.csv')
    if os.path.exists(transcript_file):
        # X_all[sample_idx] shape: (chunks, features)
        for chunk_id, chunk in enumerate(X_all[sample_idx]):
            row = {
                'participant_id': person_id,
                'chunk_id': chunk_id
            }
            # Add features with column names like feature_0, feature_1, ...
            feature_dict = {f'feature_{i}': value for i, value in enumerate(chunk)}
            row.update(feature_dict)
            rows.append(row)

        sample_idx += 1  # Move to next participant in X_all

# Convert to DataFrame
df = pd.DataFrame(rows)

# Save to CSV
df.to_csv('/content/drive/MyDrive/X_all_flat_with_ids.csv', index=False)
